##### betöltés

In [1]:
%%script False

%pip install nltk

%pip install spacy==3.7.5
%pip install hu_core_news_lg@https://huggingface.co/huspacy/hu_core_news_lg/resolve/v3.7.0/hu_core_news_lg-any-py3-none-any.whl?download=true
%pip install xx_ent_wiki_sm@https://github.com/explosion/spacy-models/releases/download/xx_ent_wiki_sm-3.7.0/xx_ent_wiki_sm-3.7.0-py3-none-any.whl?download=true

%pip install transformers sentencepiece flax
%pip install torch torchvision torchaudio

%pip install pandas
%pip install openpyxl

Couldn't find program: 'False'


In [2]:
with open('JanosVitez.txt', 'r', encoding='utf-8') as file:
    # A teljes fájl tartalmának beolvasása
    data = file.read()


fejezetek mentén elvágom

In [3]:
import re
# A reguláris kifejezés, ami a számokat keresi
pattern = r'\b([1-9]|1[0-9]|2[0-7])\b'

# A szöveg szétbontása a reguláris kifejezés mentén
data_parts = [part for part in re.split(pattern, data) if len(part) >= 100]

print(data_parts[0][:100])



Tüzesen süt le a nyári nap sugára
Az ég tetejéről a juhászbojtárra.
Fölösleges dolog sütnie oly na


##### tokenizálás

In [4]:
import nltk
from nltk import word_tokenize, sent_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BP5589\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\BP5589\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
tokenized_sentence = sent_tokenize(data_parts[0])   # mondatokra bontás
print(tokenized_sentence[:2])

['\n\nTüzesen süt le a nyári nap sugára\nAz ég tetejéről a juhászbojtárra.', 'Fölösleges dolog sütnie oly nagyon,\nA juhásznak úgyis nagy melege vagyon.']


In [6]:
tokenized_word = word_tokenize(data_parts[0])   # mondatokra bontás
print(tokenized_word[:15])

['Tüzesen', 'süt', 'le', 'a', 'nyári', 'nap', 'sugára', 'Az', 'ég', 'tetejéről', 'a', 'juhászbojtárra', '.', 'Fölösleges', 'dolog']


In [7]:
print(f'összes szó száma: {len(tokenized_word)}')
print(f'egyedi szavak száma: {len(set(tokenized_word))}')

összes szó száma: 343
egyedi szavak száma: 221


##### stopszavak eldobása

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BP5589\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
meaningful_words = [word for word in tokenized_word if word.lower() not in stopwords.words('hungarian')]

In [10]:
print(f'összes szó száma (-stop): {len(meaningful_words)}')
print(f'egyedi szavak száma (-stop): {len(set(meaningful_words))}')

összes szó száma (-stop): 265
egyedi szavak száma (-stop): 184


In [11]:
print(meaningful_words[:15])

['Tüzesen', 'süt', 'le', 'nyári', 'nap', 'sugára', 'ég', 'tetejéről', 'juhászbojtárra', '.', 'Fölösleges', 'dolog', 'sütnie', 'oly', ',']


##### Snowball stemmer

In [12]:
from nltk.stem.snowball import HungarianStemmer
stemmer = nltk.stem.SnowballStemmer('hungarian')
stemmed_word = [stemmer.stem(word) for word in meaningful_words]

In [13]:
print(f'összes szó száma (stemmed): {len(stemmed_word)}')
print(f'egyedi szavak száma (stemmed): {len(set(stemmed_word))}')

összes szó száma (stemmed): 265
egyedi szavak száma (stemmed): 167


In [14]:
print(stemmed_word[:15])

['tüzes', 'sü', 'le', 'nyár', 'nap', 'sug', 'ég', 'tetejéről', 'juhászbojtár', '.', 'fölösleges', 'dolog', 'sütn', 'oly', ',']


###### HuSpacy token

In [15]:
import spacy
nlp_hu = spacy.load("hu_core_news_lg")
#nlp_hu = spacy.load("xx_ent_wiki_sm")

In [16]:
import pandas as pd

doc = nlp_hu(data_parts[0])

meaningful_pos = {"NOUN", "VERB", "ADJ", "ADV"}

df_tokens = pd.DataFrame([[token.text, token.lemma_, token.pos_, token.is_stop, token.norm_] for token in doc if token.pos_ in meaningful_pos and token.is_alpha and not token.is_stop],   
                         columns=['Text', 'Lemme', 'POS', 'Is_Stop', 'norm'])
print(df_tokens.head(5))

print(df_tokens['Lemme'].str.lower().to_list()[:15]) 



      Text  Lemme   POS  Is_Stop     norm
0  Tüzesen  Tüzes   ADJ    False  tüzesen
1      süt    süt  VERB    False      süt
2    nyári  nyári   ADJ    False    nyári
3      nap    nap  NOUN    False      nap
4   sugára  sugár  NOUN    False   sugára
['tüzes', 'süt', 'nyári', 'nap', 'sugár', 'ég', 'tető', 'juhászbojtár', 'fölösleges', 'dolog', 'süt', 'juhász', 'úgyis', 'meleg', 'vagyon']


##### szómátrix

In [ ]:


meaningful_pos = {"NOUN", "VERB", "ADJ", "ADV"}
df_words = pd.DataFrame(columns=['lemma']).set_index('lemma')

for idx, part in enumerate(data_parts):
    doc = nlp_hu(part)
    lemmas = [token.lemma_.lower() for token in doc if token.pos_ in meaningful_pos and token.is_alpha and not token.is_stop]
    lemmas_set = list(set(lemmas))
    
    df_lemmas_set = pd.DataFrame({'lemma': lemmas_set})
    df_lemmas_set[idx] = df_lemmas_set['lemma'].apply(lambda x: lemmas.count(x))
    
    df_words = df_words.join(df_lemmas_set.set_index('lemma'), how='outer')


# Calculate the row sums and order the dataframe by these sums
df_words['row_sum'] = df_words.sum(axis=1)
df_words = df_words.sort_values(by='row_sum', ascending=False)



          0    1    2    3    4    5    6    7    8    9  ...   18    19   20  \
lemma                                                     ...                   
vitéz   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  9.0   4.0  7.0   
lát     NaN  NaN  1.0  6.0  NaN  5.0  1.0  1.0  NaN  NaN  ...  7.0   6.0  3.0   
nap     1.0  1.0  1.0  NaN  8.0  1.0  4.0  NaN  NaN  1.0  ...  2.0   1.0  2.0   
király  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  NaN  NaN  ...  NaN  11.0  NaN   
megy    1.0  1.0  NaN  1.0  1.0  NaN  5.0  2.0  1.0  1.0  ...  2.0   1.0  1.0   

         21   22   23   24   25   26  row_sum  
lemma                                          
vitéz   1.0  2.0  1.0  1.0  NaN  5.0     49.0  
lát     NaN  1.0  NaN  2.0  1.0  4.0     47.0  
nap     1.0  1.0  NaN  NaN  2.0  1.0     40.0  
király  NaN  NaN  NaN  NaN  NaN  1.0     36.0  
megy    NaN  NaN  NaN  1.0  NaN  2.0     32.0  

[5 rows x 28 columns]


In [ ]:
#df_words.to_excel('JanosVitez_words.xlsx')

##### Summarization

In [51]:
# Transpose the df_words DataFrame
df_words_transposed = df_words.T

# Create a new DataFrame to store the top 5 words for each row
top_words_df = pd.DataFrame(index=df_words_transposed.index, columns=range(5))

# Iterate over each row in the transposed DataFrame
for idx, row in df_words_transposed.iterrows():
    # Get the top 5 words with the highest values
    top_words = row.nlargest(5).index.tolist()
    # Assign the top words to the new DataFrame
    top_words_df.loc[idx] = top_words

print(top_words_df)

               0           1            2            3         4
0            jön       patak          tud         szép       víz
1           kend         szó         tesz      szegény      száj
2          gazda         tud          szó       gondol     dolog
3            lát         jut         szép          ész     könny
4            nap          tó        sötét           ég     közép
5        zsivány        élet          lát          ház      szép
6           megy         nap           ló        vezér    katona
7         magyar       tatár        sereg          nép    király
8          hideg        megy          néz        sereg      vesz
9        csillag        föld         hegy          tud       jut
10        király       török       ország      francia    magyar
11         török          ló         basa  királylyány       fut
12        király       vitéz          szó          nap     ekkép
13         világ         kis          ugy          tud      szép
14        király       vi

In [ ]:
#top_words_df.to_excel('JanosVitez_top_words.xlsx')

extractiv

In [17]:
#https://www.kaggle.com/code/itsmohammadshahid/nlp-text-summarizer-using-spacy
from spacy.lang.hu.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

def textSummarizer(text, percentage=0):
    
    # load the model into spaCy
    nlp = spacy.load('hu_core_news_lg')
    
    # pass the text into the nlp function
    doc= nlp(text)
    
    ## The score of each word is kept in a frequency table
    tokens=[token.text for token in doc]
    freq_of_word=dict()
    
    # Text cleaning and vectorization 
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation and word.is_alpha:
                if word.text not in freq_of_word.keys():
                    freq_of_word[word.text] = 1
                else:
                    freq_of_word[word.text] += 1
                    
    # Maximum frequency of word
    max_freq=max(freq_of_word.values())
    
    # Normalization of word frequency
    for word in freq_of_word.keys():
        freq_of_word[word]=freq_of_word[word]/max_freq
        
    # In this part, each sentence is weighed based on how often it contains the token.
    sent_tokens= [sent for sent in doc.sents]
    sent_scores = dict()
    for sent in sent_tokens:
        for word in sent:
            if word.text.lower() in freq_of_word.keys():
                if sent not in sent_scores.keys():                            
                    sent_scores[sent]=freq_of_word[word.text.lower()]
                else:
                    sent_scores[sent]+=freq_of_word[word.text.lower()]
    
    if percentage==0:
        len_tokens=1
    else:
        len_tokens=int(len(sent_tokens)*percentage)
    
    # Summary for the sentences with maximum score. Here, each sentence in the list is of spacy.span type
    summary = nlargest(n = len_tokens, iterable = sent_scores,key=sent_scores.get)
    
    # Prepare for final summary
    final_summary=[word.text for word in summary]
    
    #convert to a string
    summary=" ".join(final_summary)
    
    # Return final summary
    return summary

In [18]:
l1, l2 = [], [] 
for i, part in enumerate(data_parts):
    print(f'{i}', end='->')
    l1.append(i)
    l2.append(textSummarizer(part,0))

0->1->2->3->4->5->6->7->8->9->10->11->12->13->14->15->16->17->18->19->20->21->22->23->24->25->26->

absztraktív

In [19]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer

c:\GPY\Pannon_szoveg\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
def summarize_text(text: str, model_name: str = "google/mt5-small") -> str:
    # Modell és tokenizer betöltése
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = MT5ForConditionalGeneration.from_pretrained(model_name)

    # Tokenizálás és összefoglalás generálása
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=100, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary


In [21]:
l3, l4 = [], []
for i, part in enumerate(data_parts):
    print(f'{i}:', end='->')
    l3.append(i)
    l4.append(summarize_text(part)[12:])

0:->

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


1:->2:->3:->

Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


4:->5:->6:->7:->8:->9:->10:->

...

Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


16:->17:->18:->19:->20:->21:->22:->23:->

Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


24:->25:->26:->

Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


összefűzöm táblázatosan

In [22]:
df_summary = None
df_summary = pd.DataFrame()

df_summary['part'] = l1
df_summary['extractiv'] = l2
df_summary['abstractive'] = l4

df_summary['extractiv'] = df_summary['extractiv'].apply(lambda x: x.replace('\n', '').strip())

mentem excelbe az eredményt

In [ ]:
#df_summary.to_excel('JanosVitez_summary.xlsx', index=False, sheet_name='Summary')